# Evaluating Agents

上一节中我们使用LangGraph创建了第一个agent，可以帮我们回复邮件（真正回复的功能还没有实现），再继续实现更多功能之前，我们先来学习一下如何评估（evaluation）agent的效果。
![overview-img](img/overview_eval.png)

测试环节对于一个agent能否部署到生成环境有着非常重要的影响，只有通过测试，才能知道诸如回复质量、Token消耗量、延迟、分类准确性等一些列量化的指标。

我们将使用 [LangSmith](https://docs.smith.langchain.com/) 来完成这个任务，LangSmith 是 LangChain 团队开发的，用于评估和监控 LLM 应用的平台，它提供了两种主要的方式来测试agent。

Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")
import os
import sys
sys.path.append("..")

## How to run Evaluations

### Pytest / Vitest

Pytest 和 Vitest 分别是 Python 和 JavaScript 的单元测试框架，用于运行测试代码并生成测试报告，LangSmith 中集成了这些框架，可以方便的将测试报告同步到 LangSmith 中。在本项目中，我们使用 Pytest 来运行测试代码。

Pytest 语法简洁，很容易上手，而且，对于那些有复杂逻辑，很难通用形式处理的测试场景，Pytest 也提供了足够的灵活性来处理。

### LangSmith Datasets

除了开发语言本身所提供的单元测试框架外，LangSmith 还提供了自己的数据集（Datasets）管理功能，你可以通过 LangSmith 的 API 来使用这些数据集测试你的应用。
* LangSmith datasets are great for teams who are collaboratively building out their test suite. 
* You can leverage production traces, annotation queues, synthetic data generation, and more, to add examples to an ever-growing golden dataset.
* LangSmith datasets are great when you can define evaluators that can be applied to every test case in the dataset (ex. similarity, exact match accuracy, etc.)

> Note: _annotation queues_ 是一种数据标注系统使用的队列，作用是将需要标注的数据有序的分配给标注人员，并跟踪管理标注进度、质量和一致性。


## 测试用例（Test Cases）

测试永远都是从定义测试用例开始的，这一步很重要也很有挑战性，因为你必须清楚的知道你的agent要做什么，你才能定义出正确的测试用例。

这里事先定义好了一些测试用例，每个用例包含以下字段：
- `email_input`: 输入的邮件内容
- `expected_tool_calls`: 期待的工具调用结果
- `triage_output`: 期待的分类结果
- `response_criteria`: 回复的邮件内容应该满足的标准

In [11]:
%load_ext autoreload
%autoreload 2

# eval.email_dataset 中定义好了一组测试用例
from src.eval.email_dataset import email_inputs, expected_tool_calls, triage_outputs_list, response_criteria_list

test_case_ix = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


下面我们逐个来查看一下这些测试用例的内容。

In [9]:
print("Email Input:", email_inputs[test_case_ix])

Email Input: <class 'dict'> {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lance Martin <lance@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi Lance,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}


Email Input 中包含 author、to、subject、email_thread 等信息，agent的作用就是要根据这些信息来决定该如何回复邮件。

In [10]:
print("Expected Tool Calls:", expected_tool_calls[test_case_ix])

Expected Tool Calls: ['write_email', 'done']


Expected Tool Calls 中存储了agent在执行过程中依次需要执行的工具名称，在上面这个用例中，agent需要依次调用 `write_email` 和 `done` 两个工具。

In [13]:
print("Triage Output:", triage_outputs_list[test_case_ix])

Triage Output: respond


Triage Output 是用来评估分类节点效果的，对应的三种可能值分别是 ignore, notify, respond。

In [14]:
print("Response Criteria:", response_criteria_list[test_case_ix])

Response Criteria: 
• Send email with write_email tool call to acknowledge the question and confirm it will be investigated  



Response Criteria 是针对需要回复的邮件，agent所撰写的邮件应该满足什么样的要求。

**通过上面4个测试标准可以看出，我们的测试既包含端到端的结果测试，也包含特定步骤的中间过程测试。**

## Pytest 测试

基于上面介绍的内容，首先我们来使用 Pytest 测试一下agent工具调用的效果怎么样。

In [11]:
import pytest
from src.eval.email_dataset import email_inputs, expected_tool_calls
from src.utils import format_messages_string
from src.utils import extract_tool_calls
from src.email_assistant import email_assistant

from langsmith import testing as t

`email_assistant` 是我们上节内容构建的agent，也是我们要测试的对象。

In [23]:
@pytest.mark.langsmith
@pytest.mark.parametrize(
    "email_input, expected_tool_calls",
    [
        (email_inputs[0], expected_tool_calls[0]),
        (email_inputs[3], expected_tool_calls[3]),
    ]   
)
def test_email_dataset_tool_calls(email_input, expected_tool_calls):
    """Test if email processing contains expected tool calls.

    这里只测试了工具是否被正确调用，没有测试调用顺序。
    """

    # Run the email assistant
    result = email_assistant.invoke({"email_input": email_input})

    # Extract tool calls from the result
    extracted_tool_calls = extract_tool_calls(result["messages"])

    # check if all expected tool calls are in the extracted ones
    missing_calls = [call for call in expected_tool_calls if call not in extracted_tool_calls]

    t.log_outputs({
        "missing_calls": missing_calls,
        "extracted_tool_calls": extracted_tool_calls,
        "response": format_messages_string(result["messages"])
    })

    assert len(missing_calls) == 0


上述代码完整的展示了如何使用 Pytest 和 LangSmith 来测试一个智能体。有几点内容需要注意一下：
- 只需要添加装饰器 `@pytest.mark.langsmith`，就可以将测试结果自动上传到 LangSmith 进行查看；
- 通过装饰器 `@pytest.mark.parameterize`，可以将测试用例参数化。

### Running Pytest

我们需要再命令行中运行上述代码，首先我们需要将代码整理到 `tests/test_tools.py` 文件中，然后在命令行中切换到 `tests` 目录下，执行：
```bash
LANGSMITH_TEST_SUITE='Email assistant 04: Test Tools For Interrupt'  pytest test_tools.py
```
其中，`LANGSMITH_TEST_SUITE` 是一个环境变量，用于指定测试数据集的名称。

### 查看结果

运行结束之后，在 LangSmith 平台（[https://smith.langchain.com/](https://smith.langchain.com/)） 的 Datasets & Experiments 页面可以看到你的测试结果。

![langsmith_pytest.png](img/langsmith_pytest.png)

## LangSmith Datasets 测试

学习完了使用 Pytest 测试工具调用的结果之后，接着学习如何使用 LangSmith Datasets 来测试邮件分类（triage_router）的效果。

下图展示了 LangSmith Datasets 的工作流程，Dataset Examples 中的 inputs 会作为参数传入到 Agent 中（也就是`eamil_assistant`），然后将 Agent 的输出和 reference outputs 通过 Test Function 进行比较，最后输入测试结果。

![LangSmith Datasets 测试流程](img/eval_detail.png)

LangSmith 提供了对应的 SDK 来完成这些操作。

In [24]:
from langsmith import Client

# Initialize LangSmith client
client = Client()

### 定义 Dataset

要使用 LangSmith Datasets，首先需要定义一个 Dataset。并将这个 Dataset 上传到 LangSmith 服务上。

我们将提前在 `src/eval/email_dataset.py` 中定义好的一组测试用例上传到 LangSmith 上。

In [6]:
from src.eval.email_dataset import examples_triage

每个用例包含邮件输入和正确的分类结果：
```python
examples_triage = [
  {
      "inputs": {"email_input": email_input_1},
      "outputs": {"classification": triage_output_1},   # NOTE: This becomes the reference_output in the created dataset
  }, ...
]
```

In [7]:
print("Dataset Example Input (inputs):", examples_triage[0]['inputs'])

Dataset Example Input (inputs): {'email_input': {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lance Martin <lance@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi Lance,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}}


In [8]:
print("Dataset Example Reference Output (reference_outputs):", examples_triage[0]['outputs'])

Dataset Example Reference Output (reference_outputs): {'classification': 'respond'}


In [ ]:
# Dataset name
dataset_name = "04 E-Mail Triage Evaluation"

# If the dataset doesn't exist, create it
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name = dataset_name,
        description = "A dataset of e-mails and their triage decisions."
    )

    # Add examples to the dataset
    client.create_examples(dataset_id=dataset.id, examples=examples_triage)


上述代码执行完后，可以在 LangSmith 平台上看到我们创建的 Dataset。每个 Example 都包含一个 input 和 reference output。

![LangSmith Dataset](./img/04_langsmith_dataset.png)

### 目标函数（Target Function）

目标函数就是要测试的函数，这里需要测试 triage_router 的效果怎么样，所以我们需要写一个目标函数，将邮件输入到 triage_router 中，然后获取其输出。

In [12]:
def target_email_assistant(input: dict) -> dict:
    """Process an email through the workflow-based email assistant."""

    response = email_assistant.nodes['triage_router'].invoke({"email_input": input["email_input"]})
    return {"classification_decision": response.update['classification_decision']}

在具体执行的时候，LangSmith 的 API 会将 dataset 中的每一条数据，将其 `inputs` 字段取出，调用 `target_email_assistant` 函数，然后将函数的输出与 `reference_outputs` 进行比较，得到一个评估结果。

代码中有两点说明：
- `email_assistant.nodes` 可以访问到 graph 中每一个节点；
- `triage_router` 返回的是一个 Command 对象，包含 `update` 和 `goto` 两个内容，这里我们只关心 `update`，即对 State 的更新内容。

### 评估函数（Evaluator Function）

对目标函数的输出结果，应该怎么评价其好坏呢？现在我们已经有了：
* Reference outputs: `"reference_outputs": {"classification": triage_output_1} ...`
* Agent outputs: `"outputs": {"classification_decision": agent_output_1} ...`

我们希望评估 agent's output 和 reference output 之间的差异。因此，我们将定义一个评估函数来比较他们的结果。这个函数中，入参默认为：
- `outputs`: agent's output
- `reference_outputs`: reference output

In [13]:
def classification_evaluator(outputs: dict, reference_outputs: dict) -> bool:
    """Check if the answer exactly matches the expected answer."""
    return outputs["classification_decision"].lower() == reference_outputs["classification"].lower()

### 开始测试

当我们定义好所需要的函数后，怎么将他们串联起来进行测试呢？答案很简单：`evaluate` 函数会帮我们完成这些。

In [14]:
run_expt = True # set to true if you want to run the experiment
if run_expt:
    experiment_results_workflow = client.evaluate(
        target_email_assistant, # run agent
        data=dataset_name, # dataset name
        evaluators=[
            classification_evaluator
        ],
        experiment_prefix="E-mail assistant workflow", # name of the experiment
        max_concurrency=2, # number of concurrent evaluations
    )

/Users/zhengquan/miniconda3/envs/agents-course/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'E-mail assistant workflow-25fd6e5b' at:
https://smith.langchain.com/o/705cd733-208b-5894-8174-4d8f8d81a26f/datasets/047df233-6b46-4a77-82bb-9c2310c0b3d5/compare?selectedSessions=65812575-abc5-415a-b85e-34cad2be3ff7




0it [00:00, ?it/s]

🚫 Classification: IGNORE - This email can be safely ignored


1it [00:03,  3.29s/it]

...

13it [00:15,  1.06s/it]

🚫 Classification: IGNORE - This email can be safely ignored


15it [00:16,  1.32it/s]

🔔 Classification: NOTIFY - This email contains important information
🔔 Classification: NOTIFY - This email contains important information


16it [00:17,  1.08s/it]


执行结束后，我们可以在 LangSmith UI 中查看结果。

![测试结果](img/04_langsmith_dataset_eval_result.png)

## LLM-as-Judge Evaluation

上面两个部分，我们分别使用 `Pytest` 测试了工具调用的效果，使用 LangSmith `evaluate()` 测试了邮件分类的效果。如下图所示，这两部分都属于单元测试（Unit Test），且都有着结构化的标准结果。

![测试分类](img/eval_types.png)

接下来，我们要进行整个 email assistant 的最终输出结果测试，他是一个典型的端到端（End-to-End, E2E）测试，将所有的过程和结果都概括在一起进行评估。

因为 agent 的最终输出的邮件内容不固定的，所以无法像之前那样进行结构化的测试。

测试非结构化的输出，除了人为的去判断外，现在最常用的方法就是使用 LLM 作为裁判来进行评判。下面将介绍具体怎么做。

首先，需要定义一个结构化的输出用于处理 LLM 的评价结果。

In [16]:
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

class CriteriaGrade(BaseModel):
    """Score the response against specific criteria."""

    justification: str = Field(description="The justification for the grade and score, including specifict examples from the response.")
    grade: bool = Field(description="Dose the resonse meet the provided criteria?")

# Create a global LLM for evaluation to avoid recreate it for each test
model_name = os.getenv("OPENAI_MODEL")
model_provider = os.getenv("MODEL_PROVIDER")
criteria_eval_llm = init_chat_model(model_name, model_provider=model_provider)
criteria_eval_structured_llm = criteria_eval_llm.with_structured_output(CriteriaGrade)

In [18]:
from src.eval.email_dataset import email_inputs, response_criteria_list

email_input = email_inputs[0]
print("Email Input:", email_input)
success_criteria = response_criteria_list[0]
print("Success Criteria:", success_criteria)

Email Input: {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lance Martin <lance@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi Lance,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}
Success Criteria: 
• Send email with write_email tool call to acknowledge the question and confirm it will be investigated  



查看其中一条数据：
```
Success Criteria: 
• Send email with write_email tool call to acknowledge the question and confirm it will be investigated  
```
我们会发现，这个评价标准是非常口语化的，并且包含了需要调用的工具，很难用固定的规则来实现。

In [19]:
response = email_assistant.invoke({"email_input": email_input})

📧 Classification: RESPOND - This email requires a response


In [ ]:
from src.eval.prompts import RESPONSE_CRITERIA_SYSTEM_PROMPT

# Format the messages into a string for evaluation
all_messages_str = format_messages_string(response["messages"])

print(RESPONSE_CRITERIA_SYSTEM_PROMPT)

You are evaluating an email assistant that works on behalf of a user.

You will see a sequence of messages, starting with an email sent to the user. 

You will then see the assistant's response to this email on behalf of the user, which includes any tool calls made (e.g., write_email, schedule_meeting, check_calendar_availability, done).

You will also see a list of criteria that the assistant's response must meet.

Your job is to evaluate if the assistant's response meets ALL the criteria bullet points provided.

IMPORTANT EVALUATION INSTRUCTIONS:
1. The assistant's response is formatted as a list of messages.
2. The response criteria are formatted as bullet points (•)
3. You must evaluate the response against EACH bullet point individually
4. ALL bullet points must be met for the response to receive a 'True' grade
5. For each bullet point, cite specific text from the response that satisfies or fails to satisfy it
6. Be objective and rigorous in your evaluation
7. In your justificatio

`RESPONSE_CRITERIA_SYSTEM_PROMPT` 中说明了这个 llm evaluator 的工作方式。

下面的 user prompt 中会指明当前的这条数据所要满足的标准。

In [ ]:
eval_result = criteria_eval_structured_llm.invoke(
    [
        {"role": "system", "content": RESPONSE_CRITERIA_SYSTEM_PROMPT},
        {
            "role": "user",
            "content": f"""Response criteria: {success_criteria} \n\n Assistant's response: {all_messages_str} \n\n Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation."""
        }
    ]
)

执行完代码，会获取到一个 `CriteriaGrade` 对象，里面包含了评估过程和最后的结果。如果 `grade` 为 True，则表示评估通过，否则表示评估未通过。

In [23]:
print(eval_result.justification)
print(eval_result.grade)

• The assistant sent an email using the write_email tool call, as shown in the "Tool Calls:" section. The call includes the recipient's email (alice.smith@company.com), subject (Re: Quick question about API documentation), and content that acknowledges Alice's question and confirms that the issue will be investigated. This meets the criteria of sending an email to acknowledge the question and confirm it will be investigated.
True


## 更完整的评估

目前为止，我们已经尝试了：
- Pytest 单元测试
- LangSmith `evaluate()` 单元测试
- LLM as a judge 端到端测试

接下来，我们要把这些内容整理到一起，进行更全面、更丰富的评估。这部分评估代码已经整理到了 `tests/test_response.py` 中，我们来运行一下看看效果。

切换到 `tests` 目录，执行命令：
```bash
LANGSMITH_TEST_SUITE='04 Email assistant: Test Full Response Interrupt' LANGSMITH_EXPERIMENT='email_assistant' pytest test_response.py --agent-module email_assistant
```
其中：
- `LANGSMITH_TEST_SUITE`：LangSmith 中的测试数据集的名称
- `LANGSMITH_EXPERIMENT`：LangSmith 中的实验名称

整个 `test_response.py` 是使用 `Pytest` 的方法进行的，装饰器中的测试用例会自动上传到 LangSmith 中，并创建名称为 `LANGSMITH_TEST_SUITE` 的数据集。

在 `test_response.py` 中，我们定义了两个测试用例：
- `test_email_dataset_tool_calls`：测试邮件处理是否包含预期的工具调用
- `test_response_criteria_evaluation`：测试响应是否满足特定标准，其中包含了 LLM-as-a-judge 的内容


脚本执行完成之后，我们可以在 LangSmith UI 上看到执行结果。
![完整测试结果](./img/04_full_test_result.png)

同时，我们也可以通过 SDK 来获取结果到本地。

In [25]:
# TODO: Copy your experiment name here
experiment_name = "email_assistant:0346f18c"
# Set this to load expt results
load_expt = True
if load_expt:
    email_assistant_experiment_results = client.read_project(project_name=experiment_name, include_stats=True)
    print("Latency p50:", email_assistant_experiment_results.latency_p50)
    print("Latency p99:", email_assistant_experiment_results.latency_p99)
    print("Token Usage:", email_assistant_experiment_results.total_tokens)
    print("Feedback Stats:", email_assistant_experiment_results.feedback_stats)

Latency p50: 0:00:11.443000
Latency p99: 0:00:24.026950
Token Usage: 67435
Feedback Stats: {'pass': {'n': 16, 'avg': 0.6875, 'stdev': 0.46351240544347894, 'errors': 0, 'values': {}}}


上述代码中，需要注意几点：
- `experiment_name` 需要你从 UI 上 copy 下来；
- `experiment_name` 的值 "email_assistant:0346f18c" 的前缀就是运行测试命令时设置的环境变量 `LANGSMITH_EXPERIMENT` 的值